# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 3 2023 10:06AM,254144,21,637226,"NBTY Global, Inc.",Released
1,Jan 3 2023 10:05AM,254142,21,637158,"NBTY Global, Inc.",Released
2,Jan 3 2023 10:01AM,254140,19,ADV80009458,"AdvaGen Pharma, Ltd",Released
3,Jan 3 2023 10:01AM,254140,19,ADV80009470,"AdvaGen Pharma, Ltd",Released
4,Jan 3 2023 10:01AM,254140,19,ADV80009517,"AdvaGen Pharma, Ltd",Released
5,Jan 3 2023 9:47AM,254135,10,PRF-45880,Bio-PRF,Released
6,Jan 3 2023 9:47AM,254128,15,8960785,"Mizner Bioscience, LLC",Released
7,Jan 3 2023 9:39AM,254134,10,8984091,"Snap Medical Industries, LLC",Released
8,Jan 3 2023 9:39AM,254134,10,8984092,"Snap Medical Industries, LLC",Released
9,Jan 3 2023 9:39AM,254134,10,8984093,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,254134,Released,4
26,254135,Released,1
27,254140,Released,3
28,254142,Released,1
29,254144,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254134,NaN,4.0
254135,NaN,1.0
254140,NaN,3.0
254142,NaN,1.0
254144,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254090,44.0,18.0
254104,0.0,1.0
254107,1.0,0.0
254108,0.0,24.0
254111,0.0,53.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254090,44,18
254104,0,1
254107,1,0
254108,0,24
254111,0,53


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254090,44,18
1,254104,0,1
2,254107,1,0
3,254108,0,24
4,254111,0,53


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254090,44,18
1,254104,,1
2,254107,1,
3,254108,,24
4,254111,,53


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc."
1,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc."
2,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd"
5,Jan 3 2023 9:47AM,254135,10,Bio-PRF
6,Jan 3 2023 9:47AM,254128,15,"Mizner Bioscience, LLC"
7,Jan 3 2023 9:39AM,254134,10,"Snap Medical Industries, LLC"
11,Jan 3 2023 9:38AM,254133,10,"Methapharm, Inc."
31,Jan 3 2023 9:36AM,254132,19,"Emersa Waterbox, LLC"
32,Jan 3 2023 9:35AM,254127,15,"Brookfield Pharmaceuticals, LLC"
108,Jan 3 2023 9:34AM,254131,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc.",,1
1,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc.",,1
2,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd",,3
3,Jan 3 2023 9:47AM,254135,10,Bio-PRF,,1
4,Jan 3 2023 9:47AM,254128,15,"Mizner Bioscience, LLC",,1
5,Jan 3 2023 9:39AM,254134,10,"Snap Medical Industries, LLC",,4
6,Jan 3 2023 9:38AM,254133,10,"Methapharm, Inc.",,20
7,Jan 3 2023 9:36AM,254132,19,"Emersa Waterbox, LLC",,1
8,Jan 3 2023 9:35AM,254127,15,"Brookfield Pharmaceuticals, LLC",6,70
9,Jan 3 2023 9:34AM,254131,10,Methapharm-G,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc.",1,
1,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc.",1,
2,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd",3,
3,Jan 3 2023 9:47AM,254135,10,Bio-PRF,1,
4,Jan 3 2023 9:47AM,254128,15,"Mizner Bioscience, LLC",1,
5,Jan 3 2023 9:39AM,254134,10,"Snap Medical Industries, LLC",4,
6,Jan 3 2023 9:38AM,254133,10,"Methapharm, Inc.",20,
7,Jan 3 2023 9:36AM,254132,19,"Emersa Waterbox, LLC",1,
8,Jan 3 2023 9:35AM,254127,15,"Brookfield Pharmaceuticals, LLC",70,6
9,Jan 3 2023 9:34AM,254131,10,Methapharm-G,10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc.",1,
1,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc.",1,
2,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd",3,
3,Jan 3 2023 9:47AM,254135,10,Bio-PRF,1,
4,Jan 3 2023 9:47AM,254128,15,"Mizner Bioscience, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc.",1.0,NaN
1,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc.",1.0,NaN
2,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd",3.0,NaN
3,Jan 3 2023 9:47AM,254135,10,Bio-PRF,1.0,NaN
4,Jan 3 2023 9:47AM,254128,15,"Mizner Bioscience, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc.",1.0,0.0
1,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc.",1.0,0.0
2,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd",3.0,0.0
3,Jan 3 2023 9:47AM,254135,10,Bio-PRF,1.0,0.0
4,Jan 3 2023 9:47AM,254128,15,"Mizner Bioscience, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3811842,477.0,0.0
12,762362,3.0,0.0
15,762381,75.0,6.0
19,1016469,22.0,45.0
20,508233,2.0,0.0
21,508286,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3811842,477.0,0.0
1,12,762362,3.0,0.0
2,15,762381,75.0,6.0
3,19,1016469,22.0,45.0
4,20,508233,2.0,0.0
5,21,508286,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,477.0,0.0
1,12,3.0,0.0
2,15,75.0,6.0
3,19,22.0,45.0
4,20,2.0,0.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,477.0
1,12,Released,3.0
2,15,Released,75.0
3,19,Released,22.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,0.0,0.0,6.0,45.0,0.0,0.0
Released,477.0,3.0,75.0,22.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,0.0,0.0,6.0,45.0,0.0,0.0
1,Released,477.0,3.0,75.0,22.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,0.0,0.0,6.0,45.0,0.0,0.0
1,Released,477.0,3.0,75.0,22.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()